# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
import pandas as pd
import numpy as np
import glob
pd.options.mode.chained_assignment = None

DATA_FOLDER = './Data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [38]:
countries = "guinea" #countries = ["guinea", "liberia", "sl"]

path = DATA_FOLDER+"ebola/"+countries+"_data/"

files = glob.glob(path+"*.csv")
cols = ["Date","Description","Totals"]

df = pd.concat((pd.read_csv(f,usecols=cols,parse_dates=['Date']) for f in files),ignore_index=True)

In [41]:
df.index = df.Date

In [57]:
new_cases = df[df.Description=="Total new cases registered so far"].Totals.astype(str).astype(int)
deaths = df[df.Description=="Total deaths (confirmed + probables + suspects)"].Totals.astype(str).astype(int)

In [63]:
avg_new_cases = new_cases.resample('M').mean()
avg_deaths = deaths.resample('M').mean()

In [91]:
pd.DataFrame(avg_new_cases.values,avg_new_cases.index.month)
df_deaths = pd.DataFrame({"avg deaths":avg_deaths.values,"Month":avg_deaths.index.month})
df_deaths
#avg_deaths.index.month

,Month,avg deaths
0,8,543.2000
1,9,603.1875
2,10,739.0000


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [79]:
# Read MID files

path=DATA_FOLDER+"microbiome/"
MID_files = glob.glob(path+"MID*.xls")
MID_files

['./Data/microbiome/MID6.xls',
 './Data/microbiome/MID7.xls',
 './Data/microbiome/MID5.xls',
 './Data/microbiome/MID4.xls',
 './Data/microbiome/MID1.xls',
 './Data/microbiome/MID3.xls',
 './Data/microbiome/MID2.xls',
 './Data/microbiome/MID9.xls',
 './Data/microbiome/MID8.xls']

In [80]:
df_microbiome=pd.concat((pd.read_excel(f,index_col=0,header=None) for f in MID_files),axis=1)
df_microbiome.columns=['MID1','MID2','MID3','MID4','MID5','MID6','MID7','MID8','MID9']
df_microbiome=df_microbiome.transpose()
df_microbiome.index.names=['BARCODE']
df_microbiome

,"Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Acidilobaceae Acidilobus","Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Caldisphaeraceae Caldisphaera","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Stetteria","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Sulfophobococcus","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Thermodiscus","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Thermosphaera","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Hyperthermus","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Pyrodictium","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Pyrolobus",...,"Bacteria ""Verrucomicrobia"" Opitutae Puniceicoccales Puniceicoccaceae Puniceicoccus","Bacteria ""Verrucomicrobia"" Subdivision3 Subdivision3_genera_incertae_sedis","Bacteria ""Verrucomicrobia"" Verrucomicrobiae Verrucomicrobiales Verrucomicrobiaceae Persicirhabdus","Bacteria ""Verrucomicrobia"" Verrucomicrobiae Verrucomicrobiales Verrucomicrobiaceae Verrucomicrobium",Bacteria Cyanobacteria Cyanobacteria Chloroplast Bangiophyceae,Bacteria Cyanobacteria Cyanobacteria Chloroplast Chlorarachniophyceae,Bacteria Cyanobacteria Cyanobacteria Chloroplast Streptophyta,Bacteria OD1 OD1_genera_incertae_sedis,Bacteria OP11 OP11_genera_incertae_sedis,Bacteria TM7 TM7_genera_incertae_sedis
BARCODE,,,,,,,,,,,,,,,,,,,,,
MID1,NaN,NaN,7.0,NaN,1.0,NaN,NaN,NaN,1.0,2.0,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
MID2,NaN,1.0,8.0,NaN,2.0,NaN,1.0,NaN,1.0,1.0,...,NaN,4.0,9.0,39.0,1.0,2.0,NaN,NaN,NaN,NaN
MID3,5.0,26.0,28.0,1.0,5.0,NaN,2.0,NaN,2.0,3.0,...,4.0,NaN,NaN,NaN,2.0,5.0,59.0,4.0,NaN,NaN
MID4,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.0,44.0,NaN,NaN,NaN
MID5,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,2.0,85.0,1388.0,NaN,NaN,NaN
MID6,1.0,15.0,14.0,NaN,4.0,1.0,1.0,NaN,3.0,NaN,...,2.0,NaN,2.0,1.0,1.0,4.0,54.0,1.0,26.0,NaN
MID7,2.0,14.0,23.0,NaN,1.0,NaN,2.0,1.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,2.0
MID8,NaN,1.0,16.0,NaN,2.0,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,10.0,NaN,NaN,NaN
MID9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,4.0,NaN,NaN,NaN,NaN


In [81]:
df_microbiome.columns=[i for i in range(0,9)]

ValueError: Length mismatch: Expected axis has 775 elements, new values have 9 elements

In [82]:
df_metadata=pd.read_excel('Data/microbiome/metadata.xls',index_col='BARCODE')
df_metadata

,GROUP,SAMPLE
BARCODE,,
MID1,EXTRACTION CONTROL,NaN
MID2,NEC 1,tissue
MID3,Control 1,tissue
MID4,NEC 2,tissue
MID5,Control 2,tissue
MID6,NEC 1,stool
MID7,Control 1,stool
MID8,NEC 2,stool
MID9,Control 2,stool


In [83]:
df_merged=pd.merge(df_microbiome, df_metadata, left_index=True, right_index=True)
df_merged

,"Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Acidilobaceae Acidilobus","Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Caldisphaeraceae Caldisphaera","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Stetteria","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Sulfophobococcus","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Thermodiscus","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Thermosphaera","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Hyperthermus","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Pyrodictium","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Pyrolobus",...,"Bacteria ""Verrucomicrobia"" Verrucomicrobiae Verrucomicrobiales Verrucomicrobiaceae Persicirhabdus","Bacteria ""Verrucomicrobia"" Verrucomicrobiae Verrucomicrobiales Verrucomicrobiaceae Verrucomicrobium",Bacteria Cyanobacteria Cyanobacteria Chloroplast Bangiophyceae,Bacteria Cyanobacteria Cyanobacteria Chloroplast Chlorarachniophyceae,Bacteria Cyanobacteria Cyanobacteria Chloroplast Streptophyta,Bacteria OD1 OD1_genera_incertae_sedis,Bacteria OP11 OP11_genera_incertae_sedis,Bacteria TM7 TM7_genera_incertae_sedis,GROUP,SAMPLE
BARCODE,,,,,,,,,,,,,,,,,,,,,
MID1,NaN,NaN,7.0,NaN,1.0,NaN,NaN,NaN,1.0,2.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,EXTRACTION CONTROL,NaN
MID2,NaN,1.0,8.0,NaN,2.0,NaN,1.0,NaN,1.0,1.0,...,9.0,39.0,1.0,2.0,NaN,NaN,NaN,NaN,NEC 1,tissue
MID3,5.0,26.0,28.0,1.0,5.0,NaN,2.0,NaN,2.0,3.0,...,NaN,NaN,2.0,5.0,59.0,4.0,NaN,NaN,Control 1,tissue
MID4,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.0,44.0,NaN,NaN,NaN,NEC 2,tissue
MID5,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,2.0,85.0,1388.0,NaN,NaN,NaN,Control 2,tissue
MID6,1.0,15.0,14.0,NaN,4.0,1.0,1.0,NaN,3.0,NaN,...,2.0,1.0,1.0,4.0,54.0,1.0,26.0,NaN,NEC 1,stool
MID7,2.0,14.0,23.0,NaN,1.0,NaN,2.0,1.0,NaN,2.0,...,NaN,NaN,NaN,1.0,2.0,NaN,NaN,2.0,Control 1,stool
MID8,NaN,1.0,16.0,NaN,2.0,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,1.0,10.0,NaN,NaN,NaN,NEC 2,stool
MID9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,4.0,NaN,NaN,NaN,NaN,Control 2,stool


In [84]:
df_merged.fillna('unknown',inplace=True)
df_merged

,"Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Acidilobaceae Acidilobus","Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Caldisphaeraceae Caldisphaera","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Stetteria","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Sulfophobococcus","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Thermodiscus","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Thermosphaera","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Hyperthermus","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Pyrodictium","Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Pyrolobus",...,"Bacteria ""Verrucomicrobia"" Verrucomicrobiae Verrucomicrobiales Verrucomicrobiaceae Persicirhabdus","Bacteria ""Verrucomicrobia"" Verrucomicrobiae Verrucomicrobiales Verrucomicrobiaceae Verrucomicrobium",Bacteria Cyanobacteria Cyanobacteria Chloroplast Bangiophyceae,Bacteria Cyanobacteria Cyanobacteria Chloroplast Chlorarachniophyceae,Bacteria Cyanobacteria Cyanobacteria Chloroplast Streptophyta,Bacteria OD1 OD1_genera_incertae_sedis,Bacteria OP11 OP11_genera_incertae_sedis,Bacteria TM7 TM7_genera_incertae_sedis,GROUP,SAMPLE
BARCODE,,,,,,,,,,,,,,,,,,,,,
MID1,unknown,unknown,7,unknown,1,unknown,unknown,unknown,1,2,...,1,1,unknown,unknown,unknown,unknown,unknown,unknown,EXTRACTION CONTROL,unknown
MID2,unknown,1,8,unknown,2,unknown,1,unknown,1,1,...,9,39,1,2,unknown,unknown,unknown,unknown,NEC 1,tissue
MID3,5,26,28,1,5,unknown,2,unknown,2,3,...,unknown,unknown,2,5,59,4,unknown,unknown,Control 1,tissue
MID4,unknown,unknown,2,unknown,unknown,unknown,unknown,unknown,unknown,unknown,...,unknown,unknown,unknown,6,44,unknown,unknown,unknown,NEC 2,tissue
MID5,unknown,unknown,7,unknown,unknown,unknown,unknown,unknown,unknown,2,...,unknown,unknown,2,85,1388,unknown,unknown,unknown,Control 2,tissue
MID6,1,15,14,unknown,4,1,1,unknown,3,unknown,...,2,1,1,4,54,1,26,unknown,NEC 1,stool
MID7,2,14,23,unknown,1,unknown,2,1,unknown,2,...,unknown,unknown,unknown,1,2,unknown,unknown,2,Control 1,stool
MID8,unknown,1,16,unknown,2,unknown,unknown,unknown,5,unknown,...,unknown,unknown,unknown,1,10,unknown,unknown,unknown,NEC 2,stool
MID9,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,...,unknown,unknown,1,4,unknown,unknown,unknown,unknown,Control 2,stool


In [85]:
df_merged.index.is_unique

True

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here